# Experiment writing to a zarr store on the JASMIN s3 object store.

* Working with `s3fs`, not with `fsspec`
* ORIG: Cannot write large files (3D zoom level 10) - run out of mem.
* UPDATED: can write large files - need to chunk these properly first though.

In [1]:
from pathlib import Path

import fsspec
import s3fs
import xarray as xr
import zarr

In [51]:
# Good chunking is necessary to get this saving without crashing the notebook.
dsOLR = xr.open_mfdataset(
    '/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc',
    chunks={'time': 1, 'cell': 12582912 / 12},
)

/tmp/ipykernel_2595/682191614.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsOLR = xr.open_mfdataset(


In [2]:
s3cfg = dict([l.split(' = ') for l in Path('/home/users/mmuetz/.s3cfg').read_text().split('\n') if l])

In [3]:
# Internal access allowed in notebooks - no https and s3-ext.
jasmin_s3 = s3fs.S3FileSystem(
    anon=False, secret=s3cfg['secret_key'],
    key=s3cfg['access_key'],
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'}
)
jasmin_s3.ls('sim-data')

['sim-data/DYAMOND3_example_data']

In [54]:
store= s3fs.S3Map(root='s3://sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc', s3=jasmin_s3, check=False)


In [55]:
task = dsOLR.to_zarr(store=store)

In [4]:
# Good chunking is necessary to get this saving without crashing the notebook.
dsT = xr.open_mfdataset(
    '/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz10/glm.n2560_RAL3p3.apverd_20200120T??.air_temperature.hpz10.zarr',
    chunks={'time': 1, 'pressure': 5, 'cell': 12582912 / 12},
)

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_3304/1128701187.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_mfdataset(
/tmp/ipykernel_3304/1128701187.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_mfdataset(


In [5]:
store= s3fs.S3Map(root='s3://sim-data/DYAMOND3_example_data/healpix/air_temperature/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz10.zarr', s3=jasmin_s3, check=False)

In [6]:
dsT.nbytes / 1e9

22.749905148

In [7]:
# Stopped after 17m, transfered 4.3/~10G. Sure this was slower than last time...
dsT.to_zarr(store=store)

KeyboardInterrupt: 

In [ ]:
22.75e3 / 10

In [8]:
dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.nc')


/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_1909/333485658.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.nc')


In [9]:
dsT

<xarray.Dataset> Size: 12kB
Dimensions:                  (time: 5, pressure: 25, cell: 12)
Coordinates:
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
    forecast_period          (time) timedelta64[ns] 40B ...
    forecast_reference_time  datetime64[ns] 8B ...
  * cell                     (cell) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
Data variables:
    air_temperature          (time, pressure, cell) float64 12kB ...
Attributes:
    standard_name:    air_temperature
    units:            K
    um_stash_source:  m01s16i203
    source:           Data from Met Office Unified Model
    um_version:       13.5
    grid_mapping:     healpix_nested
    healpix_zoom:     0

In [2]:
s3cfg = dict([l.split(' = ') for l in Path('/home/users/mmuetz/.s3cfg').read_text().split('\n') if l])

In [3]:
# Internal access allowed in notebooks - no https and s3-ext.
jasmin_s3 = s3fs.S3FileSystem(
    anon=False, secret=s3cfg['secret_key'],
    key=s3cfg['access_key'],
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'}
)
jasmin_s3.ls('data')

['data/20200101T0000Z_ph2496.pp',
 'data/hpz0',
 'data/hpz10',
 'data/zarr_example.zarr']

In [4]:
store= s3fs.S3Map(root='s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr', s3=jasmin_s3, check=False)

In [ ]:
task = dsT.to_zarr(store=store)

In [14]:
dsT2 = xr.open_zarr(store=store)

/tmp/ipykernel_1909/336796369.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT2 = xr.open_zarr(store=store)


In [15]:
dsT2

<xarray.Dataset> Size: 12kB
Dimensions:                  (time: 5, pressure: 25, cell: 12)
Coordinates:
  * cell                     (cell) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
    forecast_period          (time) timedelta64[ns] 40B dask.array<chunksize=(5,), meta=np.ndarray>
    forecast_reference_time  datetime64[ns] 8B ...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
Data variables:
    air_temperature          (time, pressure, cell) float64 12kB dask.array<chunksize=(5, 25, 12), meta=np.ndarray>
Attributes:
    grid_mapping:     healpix_nested
    healpix_zoom:     0
    source:           Data from Met Office Unified Model
    standard_name:    air_temperature
    um_stash_source:  m01s16i203
    um_version:       13.5
    units:            K

In [17]:
dsT2.air_temperature.sum().compute()

<xarray.DataArray 'air_temperature' ()> Size: 8B
array(376112.32569437)
Coordinates:
    forecast_reference_time  datetime64[ns] 8B 2020-01-20

In [25]:
# Try writing a much larger dataset (zoom level 10).
dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz10.nc')


/tmp/ipykernel_1909/1905326407.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz10.nc')


In [20]:
# Runs out of mem with 24G, even though compute=False.
store= s3fs.S3Map(root='s3://data/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.4.air_temperature.hpz10.zarr', s3=jasmin_s3, check=False)
#task = dsT.to_zarr(store=store, compute=False)

In [17]:
# Good chunking is necessary to get this saving without crashing the notebook.
dsT = xr.open_mfdataset(
    '/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz10/glm.n2560_RAL3p3.apverd_20200120T??.air_temperature.hpz10.nc',
    chunks={'time': 1, 'pressure': 5, 'cell': 12582912 / 12},
)

/tmp/ipykernel_2595/461278671.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_mfdataset(
/tmp/ipykernel_2595/461278671.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_mfdataset(


In [18]:
dsT.chunks

Frozen({'time': (1, 1, 1, 1, 1, 1, 1, 1, 1), 'pressure': (5, 5, 5, 5, 5), 'cell': (1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 1048576)})

In [21]:
# I don't think this is necessary with good chunking... but might be helpful when we are saving much more data because it should allow for 
# parallel writes from different proces.
task = dsT.to_zarr(store=store, compute=False)

In [24]:
dsT = dsT.drop_vars(['time', 'forecast_period', 'forecast_reference_time'])

In [26]:
dsT.isel(time=[0]).to_zarr(store=store, region='auto')

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/dask/array/core.py:119: SerializationWarning: Can't decode floating point timedelta to 's' without precision loss, decoding to 'ns' instead. To silence this warning use time_unit='ns' in call to decoding function.
  c = np.asarray(c)
/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/dask/array/core.py:119: SerializationWarning: Can't decode floating point timedelta to 's' without precision loss, decoding to 'ns' instead. To silence this warning use time_unit='ns' in call to decoding function.
  c = np.asarray(c)
/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/dask/array/core.py:119: SerializationWarning: Can't decode floating point timedelta to 's' without precision loss, decoding to 'ns' instead. To silence this warning use time_unit='ns' in call to decoding function.
  c = np.asarray(c)
/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/

In [28]:
dsT.isel(time=slice(1, None)).to_zarr(store=store, region='auto')

In [29]:
dsT.isel(time=0, pressure=-1).air_temperature.mean().compute()

<xarray.DataArray 'air_temperature' ()> Size: 8B
array(287.12667868)
Coordinates:
    pressure  float32 4B 1e+03

In [30]:
dsTz10 = xr.open_zarr(store=store)

/tmp/ipykernel_2595/2785743947.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsTz10 = xr.open_zarr(store=store)


In [35]:
dsTz10.air_temperature.mean(dim=['pressure', 'cell']).compute()

<xarray.DataArray 'air_temperature' (time: 9)> Size: 72B
array([250.7051535 , 250.70911591, 250.7526272 , 250.79612698,
       250.81957318, 250.8162028 , 250.7930435 , 250.75950786,
                nan])
Coordinates:
    forecast_period          (time) timedelta64[ns] 72B NaT NaT NaT ... NaT NaT
    forecast_reference_time  datetime64[ns] 8B 2020-01-20
  * time                     (time) datetime64[ns] 72B 2020-01-20 ... 2020-01-21

In [39]:
dsTz10.isel(time=0, pressure=0).nbytes / 1e9

0.20132662

In [40]:
# Good chunking is necessary to get this saving without crashing the notebook.
dsOLR = xr.open_mfdataset(
    '/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc',
    chunks={'time': 1, 'cell': 12582912 / 12},
)

/tmp/ipykernel_2595/682191614.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsOLR = xr.open_mfdataset(


In [41]:
store= s3fs.S3Map(root='s3://data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc', s3=jasmin_s3, check=False)


In [42]:
task = dsOLR.to_zarr(store=store)

In [44]:
dsOLR.nbytes / 1e9

1.308623048

In [45]:
dsOLR2 = xr.open_zarr(store=store)

/tmp/ipykernel_2595/987760475.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsOLR2 = xr.open_zarr(store=store)


In [48]:
dsOLR2.load()

<xarray.Dataset> Size: 1GB
Dimensions:                     (cell: 12582912, time: 12)
Coordinates:
  * cell                        (cell) int64 101MB 0 1 2 ... 12582910 12582911
    forecast_period             (time) timedelta64[ns] 96B 00:01:59.999999971...
    forecast_reference_time     datetime64[ns] 8B 2020-01-20
  * time                        (time) datetime64[ns] 96B 2020-01-20T00:02:00...
Data variables:
    toa_outgoing_longwave_flux  (time, cell) float64 1GB 290.9 290.5 ... 201.1
Attributes:
    grid_mapping:     healpix_nested
    healpix_zoom:     10
    source:           Data from Met Office Unified Model
    standard_name:    toa_outgoing_longwave_flux
    um_stash_source:  m01s02i205
    um_version:       13.5
    units:            W m-2

In [49]:
dsOLR2.toa_outgoing_longwave_flux.mean()

<xarray.DataArray 'toa_outgoing_longwave_flux' ()> Size: 8B
array(235.82190961)
Coordinates:
    forecast_reference_time  datetime64[ns] 8B 2020-01-20

In [50]:
jasmin_s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'}
)
jasmin_s3.ls('s3://data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.nc/.zattrs')

PermissionError: Access Denied

## experiment with `fsspec`

* Basic access to s3 store works...
* But, I cannot get this working with `xarray` reading from a store that I've set up using `fsspec`

In [5]:
fs = fsspec.filesystem(
    's3',
    key=s3cfg['access_key'],
    secret=s3cfg['secret_key'],
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'},  # Works locally withing JASMIN/Notebook service.
)

In [6]:
fs.ls('data')

['data/20200101T0000Z_ph2496.pp', 'data/zarr_example.zarr']

In [19]:
s3_store = fsspec.get_mapper(
    "s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr",
    storage_options={
        'key': s3cfg['access_key'],
        'secret': s3cfg['secret_key'],        
        "client_kwargs": {"endpoint_url": "http://hackathon-o.s3.jc.rl.ac.uk"}  # Adjust for your S3-compatible storage
    }
)

In [23]:
s3_store = fsspec.get_mapper("s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr", fs=fs)

In [24]:
dsT3 = xr.open_zarr(s3_store)

TypeError: AioSession.__init__() got an unexpected keyword argument 'fs'